## Common code and variables for all prediction notebooks

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "cmr10"
plt.rcParams["font.size"] = 16
plt.rcParams['axes.unicode_minus'] = False

from sklearn import linear_model, preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix
import statsmodels.api as sm

import time
import pickle
import datetime

In [2]:
def change_font_size(fs):
    plt.rcParams["font.size"] = fs

In [3]:
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# Interesting regions to look at
# 2000232 = Bney Brak
# 2000166 = Tel Aviv
# 2000021 = Beit Shemesh (Highest cases in Israel)
# 2000241 = Modi'in Elit (Orthodox)
# 2000025 = Majdel Shams (Arab)
interesting_regions = [231, 165, 20, 240, 24]

In [6]:
def dump_to_pickle(file, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Saved file")

# Make predictions, with and without exponential decay

In [7]:
r = 0.5# ** 0.5
a = 1
n = 15

def exponential_weights_daily(r, a, n):
    return [a*r**(n-i-1) for i in range(n)]

def exponential_weights_weekly(r, a, n):
    return np.repeat([a*r**(n-i-1) for i in range(n)], 7)[-n:]

In [8]:
# set exp_decay_weekly or exp_decay_daily to non-zero numbers for that r-value in the above formula

def make_osa_preds(X_train, y_train, X_test, y_test, exp_decay_weekly=0, exp_decay_daily=0):
    y_preds = list()

    history_X = X_train
    history_y = y_train
    regr = linear_model.LinearRegression()
    for i in range(0, len(X_test), 1):
        if exp_decay_weekly > 0:
            regr.fit(history_X, history_y, sample_weight = exponential_weights_weekly(exp_decay_weekly, 1, len(history_X)))
        elif exp_decay_daily > 0:
            regr.fit(history_X, history_y, sample_weight = exponential_weights_daily(exp_decay_daily, 1, len(history_X)))
        else:
            regr.fit(history_X, history_y)
        yhat_1 = regr.predict(X_test[i:i+1])   
        y_preds.append(yhat_1)
        if i>=7:
            history_X = np.vstack((history_X, X_test[i-7:i+1-7]))
            try:
                history_y = np.hstack((history_y, y_test[i-7:i+1-7]))
            except:
                history_y = np.vstack((history_y, y_test[i-7:i+1-7]))
    
    return np.maximum(0.0, np.concatenate(y_preds).ravel())

In [9]:
# set exp_decay_weekly or exp_decay_daily to non-zero numbers for that r-value in the above formula

def make_dynamic_preds(X_train, y_train, X_test, y_test, exp_decay_weekly=0, exp_decay_daily=0):
    regr = linear_model.LinearRegression()
    if exp_decay_weekly > 0:
        regr.fit(X_train, y_train.values.ravel(), sample_weight = exponential_weights_weekly(exp_decay_weekly, 1, len(X_train)))
    elif exp_decay_daily > 0:
        regr.fit(X_train, y_train.values.ravel(), sample_weight = exponential_weights_daily(exp_decay_daily, 1, len(X_train)))
    else:
        regr.fit(X_train, y_train.values.ravel())
    y_pred = regr.predict(X_test)
    return np.maximum(0.0, y_pred)

# Train/Test Split

In [10]:
# Define training and testing dates
train_begin = "2020-04-01"
train_end = "2020-10-09"
test_begin = "2020-10-16"

new_train_end = "2020-10-09"
new_test_begin = "2020-10-16"
test_end_nov = "2020-11-30"
test_end_dec = "2020-12-31"

# Define Feature Sets

In [11]:
# Mobility features to use for all prediction tasks

press_7 = ['Pressure_7_Adj100k_1dayago']
press_7d = ['Diff Pressure_7_Adj100k_1dayago']
press_7e = ['Excess Pressure_7_Adj100k_1dayago']
#press_8 = ['Pressure_8_Adj100k']
#press_8d = ['Diff Pressure_8_Adj100k']
#press_8e = ['Excess Pressure_8_Adj100k']
#stops = ['Commuter_8', 'Resident_8']
im = ['Internal_Movement_7_Adj100k_1dayago']
im_d = ['Diff Internal_Movement_7_Adj100k_1dayago']
im_e = ['Excess Internal_Movement_7_Adj100k_1dayago']

In [12]:
# Incidence

activecases_1 = ['Active_Cases_7_Adj100k_7_1dayago'] 
activecases_3 = ['Active_Cases_7_Adj100k_7_1dayago', 'Active_Cases_7_Adj100k_7_2dayago', 'Active_Cases_7_Adj100k_7_3dayago'] 
activecases_6 = ['Active_Cases_7_Adj100k_7_1dayago', 'Active_Cases_7_Adj100k_7_2dayago', 'Active_Cases_7_Adj100k_7_3dayago',
             'Active_Cases_7_Adj100k_7_4dayago', 'Active_Cases_7_Adj100k_7_5dayago', 'Active_Cases_7_Adj100k_7_6dayago']


In [13]:
# Positivity

positivity_lag = ['Positivity_Rate_7_1dayago',
'Positivity_Rate_7_2dayago', 'Positivity_Rate_7_3dayago',
'Positivity_Rate_7_4dayago', 'Positivity_Rate_7_5dayago',
'Positivity_Rate_7_6dayago']
positivity_1 = ['Positivity_Rate_7_1dayago']
positivity_3 = ['Positivity_Rate_7_1dayago',
'Positivity_Rate_7_2dayago', 'Positivity_Rate_7_3dayago']
positivity_6 = ['Positivity_Rate_7_1dayago',
'Positivity_Rate_7_2dayago', 'Positivity_Rate_7_3dayago',
'Positivity_Rate_7_4dayago', 'Positivity_Rate_7_5dayago',
'Positivity_Rate_7_6dayago']

# Standardized Positivity


z_lag = ['z_score_1dayago',
'z_score_2dayago', 'z_score_3dayago',
'z_score_4dayago', 'z_score_5dayago',
'z_score_6dayago']
z_1 = ['z_score_1dayago']
z_3 = ['z_score_1dayago',
'z_score_2dayago', 'z_score_3dayago']
z_6 = ['z_score_1dayago',
'z_score_2dayago', 'z_score_3dayago',
'z_score_4dayago', 'z_score_5dayago',
'z_score_6dayago']

In [14]:
# New Tests

newtests_1 = ['New_Tests_7_Adj100k_1dayago'] 
newtests_3 = ['New_Tests_7_Adj100k_1dayago', 'New_Tests_7_Adj100k_2dayago', 'New_Tests_7_Adj100k_3dayago'] 
newtests_6 = ['New_Tests_7_Adj100k_1dayago', 'New_Tests_7_Adj100k_2dayago', 'New_Tests_7_Adj100k_3dayago',
             'New_Tests_7_Adj100k_4dayago', 'New_Tests_7_Adj100k_5dayago', 'New_Tests_7_Adj100k_6dayago']

newtests_lag = ['New_Tests_7_Adj100k_1dayago', 'New_Tests_7_Adj100k_2dayago',
'New_Tests_7_Adj100k_3dayago', 'New_Tests_7_Adj100k_4dayago', 'New_Tests_7_Adj100k_5dayago',
'New_Tests_7_Adj100k_6dayago']

In [15]:
# New Cases

newcases_1 = ['New_Cases_7_Adj100k_1dayago']
newcases_3 = ['New_Cases_7_Adj100k_1dayago', 'New_Cases_7_Adj100k_2dayago', 'New_Cases_7_Adj100k_3dayago']
newcases_6 = ['New_Cases_7_Adj100k_1dayago', 'New_Cases_7_Adj100k_2dayago', 'New_Cases_7_Adj100k_3dayago',
'New_Cases_7_Adj100k_4dayago', 'New_Cases_7_Adj100k_5dayago', 'New_Cases_7_Adj100k_6dayago']

In [16]:
positivity_label = 'Positivity_Rate_7_next7days'


pos_feature_sets = dict()
pos_feature_sets[1] = positivity_6 + press_7
pos_feature_sets[2] = positivity_6 + press_7d
pos_feature_sets[3] = positivity_6 + press_7e
pos_feature_sets[4] = positivity_6
pos_feature_sets[5] = positivity_3 + press_7
pos_feature_sets[6] = positivity_3 + press_7d
pos_feature_sets[7] = positivity_3 + press_7e
pos_feature_sets[8] = positivity_3
pos_feature_sets[9] = positivity_1 + press_7
pos_feature_sets[10] = positivity_1 + press_7d
pos_feature_sets[11] = positivity_1 + press_7e
pos_feature_sets[12] = positivity_1
pos_feature_sets[13] = positivity_1 + press_7 + im
pos_feature_sets[14] = positivity_1 + press_7d + im_d
pos_feature_sets[15] = positivity_1 + press_7e + im_e
pos_feature_sets[16] =  positivity_1 + im_d
pos_feature_sets[17] =  positivity_1 + im_e
pos_feature_sets[18] =  positivity_1 + im_d + newtests_1
pos_feature_sets[19] =  positivity_1 + im_e + newtests_1
pos_feature_sets[20] = positivity_1 + press_7 + newtests_1
pos_feature_sets[21] = positivity_1 + press_7d + newtests_1
pos_feature_sets[22] = positivity_1 + press_7e + newtests_1
pos_feature_sets[23] = press_7 + im + newtests_1
pos_feature_sets[24] = press_7 + im
pos_feature_sets[25] =  positivity_1 + im_d + newcases_1
pos_feature_sets[26] =  positivity_1 + im_e + newcases_1
pos_feature_sets[27] = positivity_1 + press_7 + newcases_1
pos_feature_sets[28] = positivity_1 + press_7d + newcases_1
pos_feature_sets[29] = positivity_1 + press_7e + newcases_1
pos_feature_sets[30] = press_7 + im + newcases_1
pos_feature_sets[31] =  positivity_3 + im_d
pos_feature_sets[32] =  positivity_3 + im_e
pos_feature_sets[33] =  positivity_3 + im_d + newtests_1
pos_feature_sets[34] =  positivity_3 + im_e + newtests_1
pos_feature_sets[35] = positivity_3 + press_7 + newtests_1
pos_feature_sets[36] = positivity_3 + press_7d + newtests_1
pos_feature_sets[37] = positivity_3 + press_7e + newtests_1
pos_feature_sets[38] =  positivity_3 + im_d + newcases_1
pos_feature_sets[39] =  positivity_3 + im_e + newcases_1
pos_feature_sets[40] = positivity_3 + press_7 + newcases_1
pos_feature_sets[41] = positivity_3 + press_7d + newcases_1
pos_feature_sets[42] = positivity_3 + press_7e + newcases_1
pos_feature_sets[43] =  positivity_6 + im_d
pos_feature_sets[44] =  positivity_6 + im_e
pos_feature_sets[45] =  positivity_6 + im_d + newtests_1
pos_feature_sets[46] =  positivity_6 + im_e + newtests_1
pos_feature_sets[47] = positivity_6 + press_7 + newtests_1
pos_feature_sets[48] = positivity_6 + press_7d + newtests_1
pos_feature_sets[49] = positivity_6 + press_7e + newtests_1
pos_feature_sets[50] =  positivity_6 + im_d + newcases_1
pos_feature_sets[51] =  positivity_6 + im_e + newcases_1
pos_feature_sets[52] = positivity_6 + press_7 + newcases_1
pos_feature_sets[53] = positivity_6 + press_7d + newcases_1
pos_feature_sets[54] = positivity_6 + press_7e + newcases_1

In [17]:
z_label = 'z_score'


z_feature_sets = dict()
z_feature_sets[1] = z_6 + press_7
z_feature_sets[2] = z_6 + press_7d
z_feature_sets[3] = z_6 + press_7e
z_feature_sets[4] = z_6
z_feature_sets[5] = z_3 + press_7
z_feature_sets[6] = z_3 + press_7d
z_feature_sets[7] = z_3 + press_7e
z_feature_sets[8] = z_3
z_feature_sets[9] = z_1 + press_7
z_feature_sets[10] = z_1 + press_7d
z_feature_sets[11] = z_1 + press_7e
z_feature_sets[12] = z_1
z_feature_sets[13] = z_1 + press_7 + im
z_feature_sets[14] = z_1 + press_7d + im_d
z_feature_sets[15] = z_1 + press_7e + im_e
z_feature_sets[16] =  z_1 + im_d
z_feature_sets[17] =  z_1 + im_e
z_feature_sets[18] =  z_1 + im_d + newtests_1
z_feature_sets[19] =  z_1 + im_e + newtests_1
z_feature_sets[20] = z_1 + press_7 + newtests_1
z_feature_sets[21] = z_1 + press_7d + newtests_1
z_feature_sets[22] = z_1 + press_7e + newtests_1
z_feature_sets[23] = press_7 + im + newtests_1
z_feature_sets[24] = press_7 + im
z_feature_sets[25] =  z_1 + im_d + newcases_1
z_feature_sets[26] =  z_1 + im_e + newcases_1
z_feature_sets[27] = z_1 + press_7 + newcases_1
z_feature_sets[28] = z_1 + press_7d + newcases_1
z_feature_sets[29] = z_1 + press_7e + newcases_1
z_feature_sets[30] = press_7 + im + newcases_1
z_feature_sets[31] =  z_3 + im_d
z_feature_sets[32] =  z_3 + im_e
z_feature_sets[33] =  z_3 + im_d + newtests_1
z_feature_sets[34] =  z_3 + im_e + newtests_1
z_feature_sets[35] = z_3 + press_7 + newtests_1
z_feature_sets[36] = z_3 + press_7d + newtests_1
z_feature_sets[37] = z_3 + press_7e + newtests_1
z_feature_sets[38] =  z_3 + im_d + newcases_1
z_feature_sets[39] =  z_3 + im_e + newcases_1
z_feature_sets[40] = z_3 + press_7 + newcases_1
z_feature_sets[41] = z_3 + press_7d + newcases_1
z_feature_sets[42] = z_3 + press_7e + newcases_1
z_feature_sets[43] =  z_6 + im_d
z_feature_sets[44] =  z_6 + im_e
z_feature_sets[45] =  z_6 + im_d + newtests_1
z_feature_sets[46] =  z_6 + im_e + newtests_1
z_feature_sets[47] = z_6 + press_7 + newtests_1
z_feature_sets[48] = z_6 + press_7d + newtests_1
z_feature_sets[49] = z_6 + press_7e + newtests_1
z_feature_sets[50] =  z_6 + im_d + newcases_1
z_feature_sets[51] =  z_6 + im_e + newcases_1
z_feature_sets[52] = z_6 + press_7 + newcases_1
z_feature_sets[53] = z_6 + press_7d + newcases_1
z_feature_sets[54] = z_6 + press_7e + newcases_1

In [18]:
activecases_label = "Active_Cases_7_Adj100k_next7days"

activecases_feature_sets = dict()
activecases_feature_sets[1] = activecases_6 + press_7
activecases_feature_sets[2] = activecases_6 + press_7d
activecases_feature_sets[3] = activecases_6 + press_7e
activecases_feature_sets[4] = activecases_6
activecases_feature_sets[5] = activecases_3 + press_7
activecases_feature_sets[6] = activecases_3 + press_7d
activecases_feature_sets[7] = activecases_3 + press_7e
activecases_feature_sets[8] = activecases_3
activecases_feature_sets[9] = activecases_1 + press_7
activecases_feature_sets[10] = activecases_1 + press_7d
activecases_feature_sets[11] = activecases_1 + press_7e
activecases_feature_sets[12] = activecases_1
activecases_feature_sets[13] = activecases_1 + press_7 + im
activecases_feature_sets[14] = activecases_1 + press_7d + im_d
activecases_feature_sets[15] = activecases_1 + press_7e + im_e
activecases_feature_sets[16] = activecases_1 + im
activecases_feature_sets[17] = activecases_6 + im_d
activecases_feature_sets[18] = activecases_6 + im_e
activecases_feature_sets[19] = activecases_6 + im
activecases_feature_sets[20] = activecases_6 + press_7 + im
activecases_feature_sets[21] = activecases_6 + press_7d + im_d
activecases_feature_sets[22] = activecases_6 + press_7e + im_e

In [19]:
newtests_label = 'NewTests_7_Adj100k_next7days'

newtests_feature_sets = dict()
newtests_feature_sets[1] = newtests_6 + press_7
newtests_feature_sets[2] = newtests_6 + press_7d
newtests_feature_sets[3] = newtests_6 + press_7e
newtests_feature_sets[4] = newtests_6
newtests_feature_sets[5] = newtests_3 + press_7
newtests_feature_sets[6] = newtests_3 + press_7d
newtests_feature_sets[7] = newtests_3 + press_7e
newtests_feature_sets[8] = newtests_3
newtests_feature_sets[9] = newtests_1 + press_7
newtests_feature_sets[10] = newtests_1 + press_7d
newtests_feature_sets[11] = newtests_1 + press_7e
newtests_feature_sets[12] = newtests_1
newtests_feature_sets[13] = newtests_1 + press_7 + im
newtests_feature_sets[14] = newtests_1 + press_7d + im_d
newtests_feature_sets[15] = newtests_1 + press_7e + im_e
newtests_feature_sets[16] = newtests_1 + im
newtests_feature_sets[17] = newtests_1 + im_d
newtests_feature_sets[18] = newtests_1 + im_e

In [20]:
newcases_label = "NewCases_7_Adj100k_next7days"

newcases_feature_sets = dict()
newcases_feature_sets[1] = newcases_6 + press_7
newcases_feature_sets[2] = newcases_6 + press_7d
newcases_feature_sets[3] = newcases_6 + press_7e
newcases_feature_sets[4] = newcases_6
newcases_feature_sets[5] = newcases_3 + press_7
newcases_feature_sets[6] = newcases_3 + press_7d
newcases_feature_sets[7] = newcases_3 + press_7e
newcases_feature_sets[8] = newcases_3
newcases_feature_sets[9] = newcases_1 + press_7
newcases_feature_sets[10] = newcases_1 + press_7d
newcases_feature_sets[11] = newcases_1 + press_7e
newcases_feature_sets[12] = newcases_1
newcases_feature_sets[13] = newcases_1 + press_7 + im
newcases_feature_sets[14] = newcases_1 + press_7d + im_d
newcases_feature_sets[15] = newcases_1 + press_7e + im_e
newcases_feature_sets[16] =  newcases_1 + im_d
newcases_feature_sets[17] =  newcases_1 + im_e
newcases_feature_sets[18] =  newcases_1 + im_d + newtests_1
newcases_feature_sets[19] =  newcases_1 + im_e + newtests_1
newcases_feature_sets[20] = newcases_1 + press_7 + newtests_1
newcases_feature_sets[21] = newcases_1 + press_7d + newtests_1
newcases_feature_sets[22] = newcases_1 + press_7e + newtests_1
newcases_feature_sets[23] = press_7 + im + newtests_1
newcases_feature_sets[24] = press_7 + im
newcases_feature_sets[25] =  newcases_1 + im_d + positivity_1
newcases_feature_sets[26] =  newcases_1 + im_e + positivity_1
newcases_feature_sets[27] = newcases_1 + press_7 + positivity_1
newcases_feature_sets[28] = newcases_1 + press_7d + positivity_1
newcases_feature_sets[29] = newcases_1 + press_7e + positivity_1
newcases_feature_sets[30] = press_7 + im + positivity_1
newcases_feature_sets[31] =  newcases_3 + im_d
newcases_feature_sets[32] =  newcases_3 + im_e
newcases_feature_sets[33] =  newcases_3 + im_d + newtests_1
newcases_feature_sets[34] =  newcases_3 + im_e + newtests_1
newcases_feature_sets[35] = newcases_3 + press_7 + newtests_1
newcases_feature_sets[36] = newcases_3 + press_7d + newtests_1
newcases_feature_sets[37] = newcases_3 + press_7e + newtests_1
newcases_feature_sets[38] =  newcases_3 + im_d + positivity_1
newcases_feature_sets[39] =  newcases_3 + im_e + positivity_1
newcases_feature_sets[40] = newcases_3 + press_7 + positivity_1
newcases_feature_sets[41] = newcases_3 + press_7d + positivity_1
newcases_feature_sets[42] = newcases_3 + press_7e + positivity_1
newcases_feature_sets[43] =  newcases_6 + im_d
newcases_feature_sets[44] =  newcases_6 + im_e
newcases_feature_sets[45] =  newcases_6 + im_d + newtests_1
newcases_feature_sets[46] =  newcases_6 + im_e + newtests_1
newcases_feature_sets[47] = newcases_6 + press_7 + newtests_1
newcases_feature_sets[48] = newcases_6 + press_7d + newtests_1
newcases_feature_sets[49] = newcases_6 + press_7e + newtests_1
newcases_feature_sets[50] =  newcases_6 + im_d + positivity_1
newcases_feature_sets[51] =  newcases_6 + im_e + positivity_1
newcases_feature_sets[52] = newcases_6 + press_7 + positivity_1
newcases_feature_sets[53] = newcases_6 + press_7d + positivity_1
newcases_feature_sets[54] = newcases_6 + press_7e + positivity_1

# Define Trend and Magnitude Accuracy for New Cases and Positivity Rate Preds

In [21]:
# Define trend accuracy
def trend_accuracy(pred, actual):
    alpha_hat = pd.Series(pred).diff().fillna(0.001)
    alpha = pd.Series(actual).diff().fillna(0.001).replace(0,0.001)
    c = pd.Series(np.sign(np.array(alpha_hat)/np.array(alpha))).clip(lower=0)
    evaluate = c[:-7] # Remove the last 7 days, for which we don't have data
    return sum(evaluate)/len(evaluate)

In [22]:
widespread = 7.0
substantial = 4.0
moderate = 1.0
minimal = 0.0

def classify_magnitude_newcases(new_cases_per_100k):
    if new_cases_per_100k >= widespread:
        return 4
    if new_cases_per_100k >= substantial:
        return 3
    if new_cases_per_100k >= moderate:
        return 2
    return 1

# Define MA, MA within 1 tier, and CM for pos rate

def confusion_matrix_nc(pred, actual):
    return cm_4tier_h(pred, actual, classify_magnitude_newcases)

def magnitude_accuracy_nc(pred, actual):
    return ma_h(pred, actual, confusion_matrix_nc)

def magnitude_accuracy_1_nc(pred, actual):
    return ma_1tier_h(pred, actual, confusion_matrix_nc)

In [23]:
def confusion_matrix_2(pred, actual):
    mag_hat = pd.Series(pred).fillna(0.001).apply(classify_magnitude)[:-7]
    mag = pd.Series(actual).fillna(0.001).apply(classify_magnitude)[:-7]
    return confusion_matrix(mag, mag_hat, labels=[1, 2, 3, 4])

def normalize_cm(cm):
    return cm / cm.sum(axis=1, keepdims=True)

In [24]:
def normalize_cm(cm):
    return cm / cm.sum(axis=1, keepdims=True)

In [25]:
widespread_positivity = 0.08
substantial_positivity = 0.05
moderate_positivity = 0.02
minimal_positivity = 0.0

def classify_magnitude_positivity(positivity_rate):
    if positivity_rate >= widespread_positivity:
        return 4
    if positivity_rate >= substantial_positivity:
        return 3
    if positivity_rate >= moderate_positivity:
        return 2
    return 1


# Define MA, MA within 1 tier, and CM for pos rate

def confusion_matrix_pr(pred, actual):
    return cm_4tier_h(pred, actual, classify_magnitude_positivity)

def magnitude_accuracy_pr(pred, actual):
    return ma_h(pred, actual, confusion_matrix_pr)

def magnitude_accuracy_1_pr(pred, actual):
    return ma_1tier_h(pred, actual, confusion_matrix_pr)

In [26]:
def classify_magnitude_overall(new_cases_per_100k, positivity_rate):
    if new_cases_per_100k >= widespread or positivity_rate >= widespread_positivity:
        return 4
    if new_cases_per_100k >= substantial or positivity_rate >= substantial_positivity:
        return 3
    if new_cases_per_100k >= moderate or positivity_rate >= moderate_positivity:
        return 2
    return 1

In [27]:
# Define magnitude accuracy
def ma_h(pred, actual, cm_handle):
    cm = cm_handle(pred, actual)
    return np.trace(cm) / np.sum(cm, axis=None)

# Define magnitude accuracy within 1 tier
def ma_1tier_h(pred, actual, cm_handle):
    cm = cm_handle(pred, actual)
    return (sum(np.diag(cm, k=1)) + sum(np.diag(cm, k=0)) + sum(np.diag(cm, k=-1))) / np.sum(cm, axis=None)

# Define magnitude accuracy directly from CM
def ma_from_cm(cm):
    return np.trace(cm) / np.sum(cm, axis=None)

# Define magnitude accuracy within 1 tier directly from CM
def ma_1tier_from_cm(cm):
    return (sum(np.diag(cm, k=1)) + sum(np.diag(cm, k=0)) + sum(np.diag(cm, k=-1))) / np.sum(cm, axis=None)


# Define confusion matrix for 5 tiers
def cm_5tier_h(pred, actual, classify_handle):
    mag_hat = pd.Series(pred).fillna(0.001).apply(classify_handle)[:-7]
    mag = pd.Series(actual).fillna(0.001).apply(classify_handle)[:-7]
    return confusion_matrix(mag, mag_hat, labels=[0, 1, 2, 3, 4])

# Define confusion matrix for 4 tiers
def cm_4tier_h(pred, actual, classify_handle):
    mag_hat = pd.Series(pred).fillna(0.001).apply(classify_handle)[:-7]
    mag = pd.Series(actual).fillna(0.001).apply(classify_handle)[:-7]
    return confusion_matrix(mag, mag_hat, labels=[1, 2, 3, 4])

In [28]:
t5_ac = 455
t4_ac = 210
t3_ac = 115
t2_ac = 45

def classify_magnitude_ac(active_cases):
    if active_cases >= t5_ac:
        return 4
    if active_cases >= t4_ac:
        return 3
    if active_cases >= t3_ac:
        return 2
    if active_cases >= t2_ac:
        return 1
    return 0

# Define MA, MA within 1 tier, and CM for active cases
def confusion_matrix_ac(pred, actual):
    return cm_5tier_h(pred, actual, classify_magnitude_ac)

def magnitude_accuracy_ac(pred, actual):
    return ma_h(pred, actual, confusion_matrix_ac)

def magnitude_accuracy_1_ac(pred, actual):
    return ma_1tier_h(pred, actual, confusion_matrix_ac)

# new cases tiering system for MA

t5_nct = 33
t4_nct = 16
t3_nct = 9
t2_nct = 3
#t5_nct = 34
#t4_nct = 15
#t3_nct = 8
#t2_nct = 2

def classify_magnitude_nct(new_cases):
    if new_cases >= t5_nct:
        return 4
    if new_cases >= t4_nct:
        return 3
    if new_cases >= t3_nct:
        return 2
    if new_cases >= t2_nct:
        return 1
    return 0


# Define MA, MA within 1 tier, and CM for active cases

def confusion_matrix_nct(pred, actual):
    return cm_5tier_h(pred, actual, classify_magnitude_nct)

def magnitude_accuracy_nct(pred, actual):
    return ma_h(pred, actual, confusion_matrix_nct)

def magnitude_accuracy_1_nct(pred, actual):
    return ma_1tier_h(pred, actual, confusion_matrix_nct)

# pos rate tiering system

t5_prt = 0.10
t4_prt = 0.07
t3_prt = 0.05
t2_prt = 0.025

def classify_magnitude_prt(pos_rate):
    if pos_rate >= t5_prt:
        return 4
    if pos_rate >= t4_prt:
        return 3
    if pos_rate >= t3_prt:
        return 2
    if pos_rate >= t2_prt:
        return 1
    return 0


# Define MA, MA within 1 tier, and CM for pos rate

def confusion_matrix_prt(pred, actual):
    return cm_5tier_h(pred, actual, classify_magnitude_prt)

def magnitude_accuracy_prt(pred, actual):
    return ma_h(pred, actual, confusion_matrix_prt)

def magnitude_accuracy_1_prt(pred, actual):
    return ma_1tier_h(pred, actual, confusion_matrix_prt)

In [29]:
widespread_dt = 1.5
substantial_dt = 4
moderate_dt = 52

def classify_magnitude_dt(doubling_time):
    if doubling_time <= widespread_dt:
        return 4
    if doubling_time <= substantial_dt:
        return 3
    if doubling_time <= moderate_dt:
        return 2
    return 1


# Define magnitude accuracy for positivity rate
def magnitude_accuracy_dt(pred, actual):
    mag_hat = pd.Series(pred).fillna(0.001).apply(classify_magnitude_dt)
    mag = pd.Series(actual).fillna(0.001).apply(classify_magnitude_dt)
    mag_diff = np.array(mag_hat) - np.array(mag)
    evaluate = mag_diff[:-7] # Remove the last 7 days, for which we don't have data
    eval_len = len(evaluate)
    return (eval_len - np.count_nonzero(evaluate))/eval_len

def confusion_matrix_dt(pred, actual):
    mag_hat = pd.Series(pred).fillna(0.001).apply(classify_magnitude_dt)[:-7]
    mag = pd.Series(actual).fillna(0.001).apply(classify_magnitude_dt)[:-7]
    return confusion_matrix(mag, mag_hat, labels=[1, 2, 3, 4])